<a href="https://colab.research.google.com/github/Alaaokaly/Practical-Data-Science-with-Python/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preproccesing


In [1]:
pip install pandas numpy nltk googletrans==4.0.0-rc1 keras tensorflow


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=464c6b9534023e44e033e7a53b8df671826ffb1612e8ed740ea8a5d75cf95036
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Succe

In [2]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import pandas as pd
import numpy as np
import re
import random
import tensorflow as tf
from nltk.corpus import wordnet
from googletrans import Translator
import sentencepiece as spm
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Attention, Masking
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Masking, Attention, Concatenate, Dense, Input, LSTM, Embedding
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard

In [4]:
alex_data= pd.read_csv('/content/MADAR.corpus.Alexandria.tsv', sep='\t')
msa_data = pd.read_csv('/content/MADAR.corpus.MSA.tsv', sep='\t')


In [5]:
def clean_arabic_text(text):
    if isinstance(text, str):  # Ensure input is a string
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = text.replace('أ', 'ا').replace('إ', 'ا').replace('آ', 'ا')  # Normalize
        text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)  # Remove diacritics
    return text


In [6]:
alex_data['sent_clean'] = alex_data['sent'].apply(clean_arabic_text)
msa_data['sent_clean'] = msa_data['sent'].apply(clean_arabic_text)

## Data Augmentation Functions

### Augmentation


In [7]:
def synonym_replacement(sentence):
    words = sentence.split()
    for i, word in enumerate(words):
        synonyms = wordnet.synsets(word)
        if synonyms:
            new_word = synonyms[0].lemmas()[0].name()  # Take the first synonym
            words[i] = new_word
    return ' '.join(words)

In [8]:
def random_deletion(sentence, p=0.2):
    words = sentence.split()
    if len(words) == 1:
        return words[0]
    remaining = list(filter(lambda w: random.random() > p, words))
    return ' '.join(remaining) if remaining else random.choice(words)

In [9]:
def random_insertion(sentence, n=2):
    words = sentence.split()
    for _ in range(n):
        new_word = random.choice(words)  # Insert a random word from the sentence
        words.insert(random.randint(0, len(words) - 1), new_word)
    return ' '.join(words)

In [10]:
def back_translation(sentence):
    translator = Translator()
    # Translate to a different language (e.g., English) and then back to Arabic
    translated = translator.translate(sentence, src='ar', dest='en').text
    return translator.translate(translated, src='en', dest='ar').text

In [11]:
augmented_sentences = []

for sentence in alex_data['sent_clean']:
    if isinstance(sentence, str):  # Check if the sentence is a string
        augmented_sentences.append(sentence)  # Original sentence
        augmented_sentences.append(synonym_replacement(sentence))  # Synonym Replacement
        augmented_sentences.append(random_deletion(sentence))  # Random Deletion
        augmented_sentences.append(random_insertion(sentence))  # Random Insertion
        augmented_sentences.append(back_translation(sentence))  # Back Translation


KeyboardInterrupt: 

In [13]:
# Create a new DataFrame for augmented sentences
augmented_df = pd.DataFrame({'sent': augmented_sentences})

In [14]:
augmented_df.to_csv('augmented_df.csv', index=False)


### post Augmentation

In [15]:
augmented_df = pd.read_csv('/content/augmented_df.csv')

In [16]:
# Combine with original data
merged_df = pd.concat([alex_data[['sent_clean']], augmented_df], ignore_index=True)


In [17]:
#merged_df = merged_df.sample(frac=0.2,random_state=42)

In [18]:
# Clean augmented data (ensure all sentences are strings)
merged_df['sent'] = merged_df['sent'].astype(str)  # Convert all entries to strings
merged_df['sent_clean'] = merged_df['sent'].apply(clean_arabic_text)
# Save augmented sentences
merged_df.to_csv('augmented_sentences.tsv', sep='\t', index=False)


## Prepare data for model training

In [19]:
# Prepare data for model training
all_sentences = list(merged_df['sent_clean'])
with open('all_sentences.txt', 'w', encoding='utf-8') as f:
    for sentence in all_sentences:
        f.write(sentence + '\n')

# Tokenization
spm.SentencePieceTrainer.train(input='all_sentences.txt', model_prefix='tokenizer', vocab_size=4490)
sp = spm.SentencePieceProcessor()
sp.load('tokenizer.model')

RuntimeError: Internal: src/trainer_interface.cc(662) [(trainer_spec_.vocab_size()) == (model_proto->pieces_size())] Vocabulary size too high (4490). Please set it to a value <= 572.

In [ ]:
# Tokenizing sentences
merged_df['sent_tokenized'] = merged_df['sent_clean'].apply(lambda x: sp.encode(x, out_type=str))


In [ ]:
# Define maximum length for padding
MAX_LEN = 25

# Padding the tokenized sentences
merged_df['sent_padded'] = pad_sequences(
    merged_df['sent_tokenized'].apply(lambda x: [sp.piece_to_id(token) for token in x]).tolist(),
    maxlen=MAX_LEN,
    padding='post',
    value=0
).tolist()

In [ ]:
# Prepare input (X) and output (Y) data
X = np.array(merged_df['sent_padded'].tolist())

# Tokenize and pad `msa_data` (target data Y)
msa_data['sent_tokenized'] = msa_data['sent_clean'].apply(lambda x: sp.encode(x, out_type=str))
msa_data['sent_padded'] = pad_sequences(
    msa_data['sent_tokenized'].apply(lambda x: [sp.piece_to_id(token) for token in x]).tolist(),
    maxlen=MAX_LEN,
    padding='post',
    value=0
).tolist()

In [ ]:
# Prepare output (Y)
Y = np.array(msa_data['sent_padded'].tolist())

# One-hot encoding the target sequences for the decoder (Y)
y_one_hot = np.zeros((len(Y), MAX_LEN, 4490))

for i, sentence in enumerate(Y):
    for j, word_id in enumerate(sentence):
        if word_id > 0:  # Ignore padding
            y_one_hot[i, j, word_id] = 1


In [ ]:
# Ensure Y has the same shape as logits except for the last dimension
y_one_hot = np.zeros((len(Y), MAX_LEN, 4490))

for i, sentence in enumerate(Y):
    for j, word_id in enumerate(sentence):
        if word_id > 0:  # Ignore padding
            y_one_hot[i, j, word_id] = 1

## Sequence-to-Sequence Model

In [ ]:
latent_dim = 256  # Latent dimensionality of the encoding space

# Encoder
encoder_inputs = Input(shape=(MAX_LEN,))
encoder_embedding = Embedding(input_dim=4490, output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(MAX_LEN,))
decoder_embedding = Embedding(input_dim=4490, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(4490, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:


# Masking Layer for Encoder Outputs
masking_layer = Masking(mask_value=0)  # Assuming padding value is 0
masked_encoder_outputs = masking_layer(encoder_outputs)

# Attention Layer
attention_layer = Attention()
attention_output = attention_layer([decoder_outputs, masked_encoder_outputs],
                                   mask=[None, masking_layer.compute_mask(encoder_inputs)])

# Concatenate attention output with decoder LSTM output
decoder_combined_context = Concatenate()([decoder_outputs, attention_output])

# Dense layer to generate the output sequence
decoder_dense = Dense(4490, activation='softmax')  # Output layer with vocabulary size
decoder_outputs = decoder_dense(decoder_combined_context)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
model.fit([X, Y], y_one_hot, batch_size=64, epochs=100, validation_split=0.2,
          callbacks=[early_stopping, tensorboard_callback])

In [ ]:
# Save the model
model.save('arabic_dialect_translation_model.h5.keras')